In [1]:
from matplotlib.patches import Patch
import sys, time, math
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
sys.path.append('/home/groups/ZuckermanLab/jalim/instalLocal/celltraj/celltraj')
import sctmCP_Dec15_2023 as cellTraj
import nucSCTM_CP as nucMsksCCR
from csaps import csaps
import string, pickle 
from datetime import date
from skimage import io, measure
from IPython.display import clear_output
from matplotlib.animation import FuncAnimation, FFMpegWriter
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

/home/groups/ZuckermanLab/jalim/instalLocal/anaconda3/envs/celltraj/lib/python3.9/site-packages/deeptime/util/platform.py:13: RuntimeWarning: There was a problem importing torch, treating it as unavailable. Stacktrace: libcufft.so.10: cannot open shared object file: No such file or directory
  warnings.warn(f"There was a problem importing {modname}, treating it as unavailable. Stacktrace: {e}",
/home/groups/ZuckermanLab/jalim/instalLocal/anaconda3/envs/celltraj/lib/python3.9/site-packages/deeptime/util/platform.py:13: RuntimeWarning: There was a problem importing torch, treating it as unavailable. Stacktrace: libcufft.so.10: cannot open shared object file: No such file or directory
  warnings.warn(f"There was a problem importing {modname}, treating it as unavailable. Stacktrace: {e}",
2024-09-18 12:28:16.978889: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-18 12:28:18.040646: I external/local_tsl/tsl/cuda/cu

In [2]:
trajl = 1 # Trajectory length for the morphodynamical embedding 
today = date.today()
date2day = today.strftime("%b%d-%Y")
sysName = 'LI204601_P'
figid = sysName+'_tlen'+str(trajl)+'_'+date2day
#conditions = ['A1','A2','A3','A4','A5','B1','B2','B3','B4','B5','C1','C2','C3'] # LIGANDS (CONDITIONS)
#tmSet = ['OSM1','EGF1','EGF+TGFB1','TGFB1','PBS1','OSM2','EGF2','EGF+TGFB2','TGFB2','PBS2',
#         'OSM+EGF+TGFB','OSM+EGF','OSM+TGFB']
conditions = ['B2'] # LIGANDS (CONDITIONS)
tmSet = ['EGF2']
nConditions = len(tmSet) # Total number of Ligand Conditions
inds_tmSet = [i for i in range(nConditions)]
inds_tmSet = np.array(inds_tmSet).astype(int)
nfovs = 1
fovs = 4
dateSet = ['']
pathSet = ['/home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/segsCellPose/bayesianTrackTest/']
imagingSet = [0 for i in range(nConditions)]
modelList = [None]*(nfovs*(nConditions))
modelList_conditions = np.zeros(nfovs*(nConditions)).astype(int)

In [3]:
i = 0
icond = 0
for cond in conditions:
    modelList_conditions[i] = icond
    modelList[i] = f"{pathSet[imagingSet[icond]]}{sysName}_{cond}_{fovs}{dateSet[imagingSet[icond]]}"
    #print("Models: ",modelList[i])
    icond = icond + 1

In [4]:
nmodels = len(modelList)
modelSet = [None]*nmodels
indgood_models = np.array([]).astype(int)

for i in range(nmodels):
    try:
        objFile = modelList[i]+'.obj'
        objFileHandler = open(objFile,'rb')
        modelSet[i] = pickle.load(objFileHandler)
        print('loaded '+objFile+' with '+str(modelSet[i].cells_indSet.size)+' cells')
        objFileHandler.close()
        test = len(modelSet[i].linSet)
        indgood_models = np.append(indgood_models,i)
    except:
        print("ERROR in reading *.obj files")
        sys.exit(0)

loaded /home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/segsCellPose/bayesianTrackTest/LI204601_P_B2_4.obj with 46096 cells


In [5]:
# Total number of frames (image snapshots) in one condition per FOVs
nframes = 193 
cellnumber_stdSet = np.ones(nmodels)*np.inf
# range of frame indices where cell numbers are higher: ~70-98%
sframe = 70.*nframes/100.; sframe = math.ceil(sframe)
eframe = 98.5*nframes/100.; eframe = math.ceil(eframe)
cellnumber_frames = np.arange(sframe, eframe).astype(int)
cellnumber_std_cut = .50 # This was set to 0.10 by Jeremy 
frames = np.arange(nframes)
# Abscissas at which smoothing will be done using CSAPS package
abSmooth = np.linspace(frames[0], frames[-1], 10000)

for i in indgood_models:
    ncells = np.zeros(nframes)
    ncells_smooth = np.zeros_like(ncells)
    for iS in range(nframes):
        ncells[iS]=np.sum(modelSet[i].cells_frameSet == iS)
    # Cubic Spline Approximation (CSAPS) to smoothen the data
    splfov = csaps(frames, ncells/ncells[0], abSmooth, smooth = 0.98) # Scaled by ncells[0] to avoid large numbers
    ncells_smooth = splfov*ncells[0] # smoothened cell numbers reverse scaled back to original
    cellnumber_std = np.std(ncells[cellnumber_frames] - ncells_smooth[cellnumber_frames])/np.mean(ncells[cellnumber_frames])
    cellnumber_stdSet[i] = cellnumber_std # Standard Deviation in Cell Numbers

In [6]:
indhigh_std = np.where(cellnumber_stdSet > cellnumber_std_cut)[0]
indgood_models = np.setdiff1d(indgood_models, indhigh_std)
#indgood_models = np.setdiff1d(indgood_models, np.array([51]).astype(int)) #missing comdx for EGF0_4

# get cell counts
nf = len(tmSet)
inds_tmSet_models = np.zeros(nmodels).astype(int)
inds_imagingSet_models = np.zeros(nmodels).astype(int)
i = 0
icond = 0
for cond in conditions:
    inds_tmSet_models[i] = inds_tmSet[icond]
    inds_imagingSet_models[i] = imagingSet[icond]
    icond = icond + 1

In [7]:
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        modelSet[i].Xf[np.isnan(modelSet[i].Xf)] = 0.0 #just replace with zeros for now? Not sure best...

nfeat_com = 3
Xf_com0 = np.zeros((0, nfeat_com))
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        Xf_com0 = np.append(Xf_com0,modelSet[i].Xf_com, axis = 0)

av_dx = np.nanmean(Xf_com0[:, 0])
std_dx = np.nanstd(Xf_com0[:, 0])
for i in indgood_models:
    modelSet[i].Xf_com[:, 0] = (modelSet[i].Xf_com[:, 0] - av_dx)/std_dx

In [8]:
wctm = cellTraj.Trajectory() # import Trajectory object 

nfeat = modelSet[indgood_models[0]].Xf.shape[1]
Xf = np.zeros((0, nfeat))
indtreatment = np.array([])
indcellSet = np.array([])
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        Xf = np.append(Xf, modelSet[i].Xf, axis = 0)
        indtreatment = np.append(indtreatment, i*np.ones(modelSet[i].Xf.shape[0]))
        indcellSet = np.append(indcellSet, modelSet[i].cells_indSet)

indtreatment = indtreatment.astype(int)
indcellSet = indcellSet.astype(int)

In [9]:
# Use the sklearn package (intended for ease of use over performance/scalability)
varCutOff = 10 # changed from 10 to 6 to match with JC's notebook
from sklearn.decomposition import PCA 
# n_components specifies the number of principal components to extract from the covariance matrix
pca = PCA(n_components = varCutOff) 
pca.fit(Xf) # builds the covariance matrix and "fits" the principal components
Xpca = pca.transform(Xf) # transforms the data into the pca representation
nPCs = Xpca.shape[1]

In [10]:
wctm.Xpca = Xpca
wctm.pca = pca
for i in indgood_models:
    if inds_imagingSet_models[i] == 0:
        indsf = np.where(indtreatment == i)[0]
        modelSet[i].Xpca = Xpca[indsf, :]

indgood_models = indgood_models[np.where(inds_imagingSet_models[indgood_models] == 0)[0]]

In [11]:
self = wctm
#wctm.trajl = trajl
all_trajSet = [None]*nmodels
for i in indgood_models:
    #modelSet[i].get_unique_trajectories(verbose = True)
    modelSet[i].get_unique_trajectories()

tracked cell 45973, 44 tracks, 42200 left
tracked cell 45927, 83 tracks, 40900 left
tracked cell 45826, 38 tracks, 37200 left
tracked cell 45754, 27 tracks, 34700 left
tracked cell 45664, 7 tracks, 30600 left
tracked cell 45336, 1 tracks, 30200 left
tracked cell 44685, 1 tracks, 29900 left
tracked cell 44561, 1 tracks, 29800 left
tracked cell 43621, 1 tracks, 29100 left
tracked cell 42941, 44 tracks, 28700 left
tracked cell 40409, 1 tracks, 26500 left
tracked cell 40057, 1 tracks, 26400 left
tracked cell 38518, 2 tracks, 25900 left
tracked cell 36392, 1 tracks, 24600 left
tracked cell 35580, 1 tracks, 24200 left
tracked cell 33466, 89 tracks, 23000 left
tracked cell 32837, 11 tracks, 22500 left
tracked cell 31823, 58 tracks, 22200 left
tracked cell 23996, 1 tracks, 16000 left
tracked cell 23163, 45 tracks, 15400 left
tracked cell 22760, 22 tracks, 15000 left
tracked cell 22608, 1 tracks, 14900 left
tracked cell 22162, 1 tracks, 14300 left
tracked cell 18561, 1 tracks, 12700 left
tracke

In [12]:
for i in indgood_models:
    # Set of all single-cell trajectories with absolute (irrespective of frames) cell indices
    num_single_cell_trajs = len(modelSet[i].trajectories)
    single_cell_traj_lengths = []
    for itraj in range(num_single_cell_trajs):
        single_cell_traj_inds = modelSet[i].trajectories[itraj] # Indices of single-cell trajectory across frames
        single_cell_traj_length = len(single_cell_traj_inds) # Length of a single-cell trajectory
        single_cell_traj_lengths.append(single_cell_traj_length)  
    indtrajs = np.argsort(single_cell_traj_lengths) # Length ordered array of unique single-cell trajectories 

In [13]:
############ Find the longest single-cell trajectory ############
longest_trajectory = max(single_cell_traj_lengths) 
if longest_trajectory > nframes: 
    print("Error: longest single-cell trajectory can't exceed total number of frames")
    sys.exit(0)
#print("Longest single-cell trajectory length: ",longest_trajectory)
ind_longest_trajectory = np.argmax(single_cell_traj_lengths) # Index of the longest single-cell trajectory
#print("Index of longest single-cell trajectory: ",ind_longest_trajectory)
cell_inds_longest_traj = modelSet[i].trajectories[ind_longest_trajectory] # Cell indices
#print("Cell indices of the longest single-cell trajectory: ",cell_inds_longest_traj)

In [14]:
# Import single-cell trajectory model from cell-cycle reporter' class
sctm = nucMsksCCR.nucCellMasksCCRimgs()
sctm.visual = False
start_frame = 0
sysName = 'LI204601_G'
filepath = "/home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/unNormReporterImgs/" 

In [15]:
model = f"{filepath}{sysName}_{conditions[0]}_{fovs}.h5"
print("Model name:", model)
# Load cell cycle reporter and nuclear images and nuclear masks
print('initializing...')
sctm.initialize(model, sysName)
# Get all the frames (193 at intervals of 15 minutes) for each model (condition)
sctm.get_frames()
end_frame = sctm.maxFrame
# Get the cell-cycle & nuclear reporter images & nuclear mask
sctm.get_imageSet(start_frame, end_frame)

Model name: /home/groups/ZuckermanLab/jalim/LI204601_INCUCYTE/unNormReporterImgs/LI204601_G_B2_4.h5
initializing...
Frame 0 has 1 images...
Frame 1 has 1 images...
Frame 2 has 1 images...
Frame 3 has 1 images...
Frame 4 has 1 images...
Frame 5 has 1 images...
Frame 6 has 1 images...
Frame 7 has 1 images...
Frame 8 has 1 images...
Frame 9 has 1 images...
Frame 10 has 1 images...
Frame 11 has 1 images...
Frame 12 has 1 images...
Frame 13 has 1 images...
Frame 14 has 1 images...
Frame 15 has 1 images...
Frame 16 has 1 images...
Frame 17 has 1 images...
Frame 18 has 1 images...
Frame 19 has 1 images...
Frame 20 has 1 images...
Frame 21 has 1 images...
Frame 22 has 1 images...
Frame 23 has 1 images...
Frame 24 has 1 images...
Frame 25 has 1 images...
Frame 26 has 1 images...
Frame 27 has 1 images...
Frame 28 has 1 images...
Frame 29 has 1 images...
Frame 30 has 1 images...
Frame 31 has 1 images...
Frame 32 has 1 images...
Frame 33 has 1 images...
Frame 34 has 1 images...
Frame 35 has 1 imag

In [16]:
def unmask_data_from_images(image):
    if np.ma.is_masked(image): # If the image is a masked Numpy array
         data = image.data
    else: 
        data = image
    return image

In [17]:
def cross_correlation(cropped_reporter_image, cropped_nuclear_image):

     cropped_reporter_image = unmask_data_from_images(cropped_reporter_image)
     cropped_nuclear_image = unmask_data_from_images(cropped_nuclear_image)
     
     # Flatten the images if they are 2D
     if cropped_reporter_image.ndim == 2:
         cropped_reporter_image = cropped_reporter_image.flatten()
     if cropped_nuclear_image.ndim == 2:
         cropped_nuclear_image = cropped_nuclear_image.flatten()
 
     # Compute correlation coefficient
     correlation_coefficient = np.corrcoef(cropped_reporter_image, cropped_nuclear_image)[0, 1]
 
     return correlation_coefficient

In [18]:
def nuclei_within_cell_boundary(cropped_nuclear_mask, cropped_reporter_image,
                                cell_mask, hole_pixels = 3):
    # Apply closing to the nuclear mask 
    closed_nuclear_mask = closing(cropped_nuclear_mask, square(hole_pixels))

    # Label the regions in the closed nuclear mask
    labeled_nuclear_mask = label(closed_nuclear_mask)
    regions = regionprops(labeled_nuclear_mask, intensity_image = cropped_reporter_image)
    
    # Create a mask for only the nuclei within the cell boundary
    nuclei_within_cell_mask = np.zeros_like(cropped_nuclear_mask, dtype = bool)

    for region in regions:
        # Check if the labeled region centroid is within the cell mask
        if cell_mask[int(region.centroid[0]), int(region.centroid[1])]:
            # Combine masks using logical OR to keep the result as boolean
            nuclei_within_cell_mask = np.logical_or(nuclei_within_cell_mask, labeled_nuclear_mask == region.label)

    # Ensure the mask is a boolean type
    nuclei_within_cell_mask = nuclei_within_cell_mask.astype(bool)

    return nuclei_within_cell_mask

In [19]:
# Calculate Average Signal Intensity within Cytoplasm/Nuclear Regions
def mean_intensity(image_region):
    
    sum_value = np.nansum(image_region)
    count = np.sum(~np.isnan(image_region)).astype(int)
    if count == 0:
        mean_value = np.nan
    else:
        mean_value = sum_value / count
    
    return mean_value

In [20]:
def nucByCytoRatioSingleCell(cropped_reporter_image, nuclei_within_cell_mask, cell_mask):

    # Apply the nuclei mask to the reporter image
    nucleus_region = np.ma.masked_array(cropped_reporter_image, mask=np.logical_not(nuclei_within_cell_mask))
    cyto_region = np.ma.masked_array(cropped_reporter_image, mask=np.logical_and(nuclei_within_cell_mask, cell_mask))

    average_intensity_nuc = mean_intensity(nucleus_region)
    average_intensity_cyto = mean_intensity(cyto_region)

    if np.isfinite(average_intensity_nuc) and np.isfinite(average_intensity_cyto):
        nuc_by_cyto = average_intensity_nuc / average_intensity_cyto
    else:
        nuc_by_cyto = np.nan
        print("Cannot calculate Nuc/Cyto Ratio.")
    #print(f"Nucleus intensity: {average_intensity_nuc}, Cytoplasmic intensity: {average_intensity_cyto}, Ratio : {nuc_by_cyto}")
    
    return nuc_by_cyto, cyto_region, nucleus_region

In [21]:
def get_single_cell_borders(self, indcells=None, bordersize=10):
     if not hasattr(self,'fmskSet'):
            print("Foreground masks not found: provide or derive them")
            sys.exit(0)

     nx = self.imgSet.shape[1]
     ny = self.imgSet.shape[2];
     if indcells is None:
        indcells = np.arange(self.cells_indSet.size).astype(int)
     ncells = indcells.size
     cellborder_imgs = [None]*ncells
     cellborder_msks = [None]*ncells
     cellborder_fmsks = [None]*ncells
     ii = 0
     ic = indcells
     sys.stdout.write('extracting cellborders from frame '+str(self.cells_frameSet[ic])+' image '+str(self.cells_imgfileSet[ic])+'\n')
     img = self.imgSet[self.cells_indimgSet[ic]]
     msk = self.mskSet[self.cells_indimgSet[ic]]
     fmsk = self.fmskSet[self.cells_indimgSet[ic]]
     cellblocks = self.get_cell_blocks(msk)

     xmin = np.max(np.array([cellblocks[self.cells_indSet[ic], 0, 0] - bordersize, 0]))
     xmax = np.min(np.array([cellblocks[self.cells_indSet[ic], 0, 1] + bordersize, nx - 1]))
     ymin = np.max(np.array([cellblocks[self.cells_indSet[ic], 1, 0] - bordersize, 0]))
     ymax = np.min(np.array([cellblocks[self.cells_indSet[ic], 1, 1] + bordersize, ny - 1]))
 
     imgcell = img[xmin:xmax, :]
     imgcell = imgcell[:, ymin:ymax]
     mskcell = msk[xmin:xmax, :]
     mskcell = mskcell[:, ymin:ymax]
     fmskcell = fmsk[xmin:xmax, :]
     fmskcell = fmskcell[:, ymin:ymax]

     tightmskcell = msk[cellblocks[self.cells_indSet[ic], 0, 0]:cellblocks[self.cells_indSet[ic], 0, 1], :]
     tightmskcell = tightmskcell[:, cellblocks[self.cells_indSet[ic], 1, 0]:cellblocks[self.cells_indSet[ic], 1, 1]]
     (values, counts) = np.unique(tightmskcell[np.where(tightmskcell > 0)], return_counts = True)
     icell = values[np.argmax(counts)].astype(int)
     mskcell = mskcell == icell
 
     cellborder_imgs[ii] = imgcell.copy()
     cellborder_msks[ii] = mskcell.copy()
     cellborder_fmsks[ii] = fmskcell.copy()
     self.cellborder_imgs = cellborder_imgs
     self.cellborder_msks = cellborder_msks
     self.cellborder_fmsks = cellborder_fmsks
 
     self.xmin = xmin
     self.ymin = ymin
     self.xmax = xmax
     self.ymax = ymax

In [22]:
# Get frame indices where Microscopy imaging captures bad snaps 
model_indc = 0
n_cells_prev = 0
swap_imgs_frame_inds = []
for frame_indc in range(nframes): 
    # If the number of cells is equal in two consecutive frames
    n_cells_curr = np.sum(modelSet[model_indc].cells_frameSet == frame_indc)
    if n_cells_curr == n_cells_prev:
        swap_imgs_frame_inds.append(frame_indc)
    n_cells_prev = n_cells_curr

In [23]:
def update(itt):
    swap_frames = False # Disable swapping by default 
    clear_output(wait = True)
    for ax in axs:
        ax.clear()
    indcells = np.array([cell_traj[itt]])
    fid = modelSet[model_indc].cells_frameSet[indcells[0]] # Frame index 
    get_single_cell_borders(modelSet[model_indc], indcells = indcells[0], bordersize = border_margin)
    imgcell = modelSet[model_indc].cellborder_imgs[0]
    mskcell = modelSet[model_indc].cellborder_msks[0]
    fmskcell = modelSet[model_indc].cellborder_fmsks[0]
    # Get cell-cell (CC) and cell-sorroundings (CS) borders
    ccborder, csborder = modelSet[model_indc].get_cc_cs_border(mskcell, fmskcell)
    
    # Combine border information with cell interior to create a full mask for the entire cell
    cell_mask = np.logical_or(np.logical_or(ccborder, csborder), mskcell)
    
    reporter_image = sctm.imgSet[fid]
    nuclear_image = sctm.nucImgSet[fid]
    nuclear_mask = sctm.nmskSet[fid]
 
    swap_frames = fid in swap_imgs_frame_inds # If frame index if found in "swap_imgs_frame_inds"
    if swap_frames:
        # If bad imaging, swap Cell-cycle and Nuclear channel images and nuclear masks with the previous frame
        reporter_image = sctm.imgSet[fid-1]
        nuclear_image = sctm.nucImgSet[fid-1]
        nuclear_mask = sctm.nmskSet[fid-1] 
    else:
        reporter_image = sctm.imgSet[fid]
        nuclear_image = sctm.nucImgSet[fid]
        nuclear_mask = sctm.nmskSet[fid]
    
    reporter_image = np.array(reporter_image)
    nuclear_image = np.array(nuclear_image)

    x_min, x_max = modelSet[model_indc].xmin, modelSet[model_indc].xmax
    y_min, y_max = modelSet[model_indc].ymin, modelSet[model_indc].ymax

    # Crop reporter images, nuclear images, and masks according to single-cell coordinates from cell mask
    cropped_reporter_image = reporter_image[x_min:x_max, y_min:y_max]
    cropped_nuclear_image = nuclear_image[x_min:x_max, y_min:y_max]
    cropped_nuclear_mask = nuclear_mask[x_min:x_max, y_min:y_max]

    # Mask out (ignore) the pixels that don't fit within the cell boundary
    cropped_reporter_image = np.ma.masked_where(cell_mask == 0, cropped_reporter_image)
    cropped_nuclear_image = np.ma.masked_where(cell_mask == 0, cropped_nuclear_image)
    
    # Calculate cross-correlation between cell-cycle reporter and nuclear channel images 
    cross_corr_frame = cross_correlation(cropped_reporter_image, cropped_nuclear_image)
    # Get Nuclei that are within the boundary of cell cytoplasm 
    nuclei_within_cell_mask = nuclei_within_cell_boundary(cropped_nuclear_mask, cropped_reporter_image,
                                                          cell_mask, hole_pixels = 3)
    # Compute nuc/Cyto ratio and return those regions along with it 
    nuc_by_cyto, cyto_region, nucleus_region = nucByCytoRatioSingleCell(cropped_reporter_image,
                                                                        nuclei_within_cell_mask, cell_mask)
    print(f"Ratio of Nuclear to Cyto Intensity: {nuc_by_cyto}")
    cross_corr.append(cross_corr_frame)
    ratio_nbc.append(nuc_by_cyto)
    frames_traj.append(fid)

    axs[0].imshow(cell_mask, cmap=plt.cm.gray)
    axs[0].imshow(cyto_region, cmap=plt.cm.viridis, alpha=0.7)
    axs[0].imshow(nucleus_region, cmap=plt.cm.viridis, alpha=0.7)
    axs[0].axis('off')
    
    axs[1].plot(frames_traj, ratio_nbc, color = 'b', marker = 'o', markerfacecolor = 'w', markersize = 8)
    #axs[1].plot(frames_traj, cross_corr, color = 'r', marker = 'o', markerfacecolor = 'w', markersize = 8, label = labels[1])
    axs[1].set_xlabel('Frames')
    axs[1].set_ylabel('nuc/cyto ratio')
    axs[1].tick_params(axis = 'y', labelcolor = 'b')

    # Plot cross_corr on the right y-axis
    ax2.plot(frames_traj, cross_corr, color = 'r', marker = 'o', markerfacecolor = 'w', markersize = 8)
    ax2.tick_params(axis = 'y', labelcolor = 'r')
    # Adding legends
    #axs[1].legend(loc = 'best', shadow = True, fancybox = True)
     
    # After updating the content, reapply the tight layout
    fig.tight_layout()
    return axs

## Save An Animation: Plot of cell coordinates along a single-cell trajectory

In [29]:
tid = 15 # Trajectory number from the end of the array 
save_file_indc = indtrajs[-tid]
print(f"Main Trajectory Index: {save_file_indc}")
cell_traj = modelSet[model_indc].trajectories[indtrajs[-tid]]
labels = ['Ratio Nuc/Cyto Signal Intensity', 'CC Values']
border_margin = 2
n_trajs = cell_traj.size # Number of single-cell trajectory snippets
fig, axs = plt.subplots(1, 2, figsize = (7, 4))
fig.tight_layout()  # Adjust subplots to fit into the figure area

# Create the secondary y-axis for axs[1] before the loop
ax2 = axs[1].twinx()
ax2.set_ylabel('CC Values', color = 'r')

frames_traj = []
ratio_nbc = []
cross_corr = []

########################### CALL TO SAVE IMAGES AS AN ANIMATION ###########################
# Configure the FFMpeg writer
#writer = FFMpegWriter(fps = 6, codec = 'libx264', bitrate = 5000)
try:
    anim = FuncAnimation(fig, update, frames = n_trajs, interval = 330)
    save_file = f"ratioNbC_CCTrj_{tmSet[0]}_{save_file_indc}.gif"
    #save_file = 'single_cell_trajectory_EGF'+str(save_file_indc)+'.mp4'
    anim.save(save_file, writer = 'pillow')
    #anim.save(save_file, writer = writer, dpi = 300)
    print(f"Animation saved as {save_file}")
except Exception as e:
    print(f"Error occurred: {e}")
plt.close(fig)

extracting cellborders from frame 136 image 0
Ratio of Nuclear to Cyto Intensity: 2.447314841149232
Animation saved as ratioNbC_CCTrj_EGF2_1727.gif
